<div align="center">

# eqc-models Brief

<h4>
  Wesley Dyk<br>
  <small style="font-weight: normal;">
    Senior Quantum Solutions Architect<br>
    Quantum Computing Inc.
  </small>
</h4>

<br>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/qci-wdyk/eqc-models-tutorial/blob/main/tutorial02-eqc-models.ipynb)

</div>



## Modeling-first Pattern


<img style="display: block; margin-left: auto; margin-right: auto;" src="images/model-operator-solver.png" />

**Solvers call for specific operators.**

<img style="display: block; margin-left: auto; margin-right: auto;" src="images/operator-solver.png" />

## Important Base Classes from eqc-models

- `EQCModel`
- `QuadraticModel` - offers both QUBO and Polynomial operators
- `PolynomialModel` - offers both QUBO and Polynomial operators
- `ConstraintsMixIn`
- `InequalityConstraintMixin`


## Imports

In [1]:
!pip install eqc_models
import os
import numpy as np
from eqc_models.base import QuadraticModel
from eqc_models.solvers import Dirac1CloudSolver
from eqc_models.base.constraints import ConstraintsMixIn
from eqc_models.base.polynomial import PolynomialModel
try:
    from google.colab import userdata
except ImportError:
    userdata = None
class ConstraintExample(ConstraintsMixIn, PolynomialModel):
    def __init__(self, coefficients, indices, lhs, rhs):
        self.constraints = lhs, rhs
        super(ConstraintExample,self).__init__(coefficients, indices)


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


## API Keys

In [2]:
# Define the API URL and token  for QCI
api_url ="https://api.qci-prod.com"
if userdata is None:
    api_token = "" # replace or use environment variables to configure
else:
    api_token = userdata.get("QCI_TOKEN")
    os.environ["QCI_TOKEN"] = api_token
    os.environ["QCI_API_URL"] = api_url

## Intended Usage

`EQCModel` is an "abstract" class which defines some key interfaces. 
`QuadraticModel` and `PolynomialModel` are useful on their own, but the can also be used to build much more useful objects. 

Uses the **mixin design pattern** to incorporate functionality that can be shared across models and solvers.


### `QuadraticModel`

$$
E(x)=\sum_i C_ix_i + \sum_{ij} J_{ij}x_ix_j
$$

Specify a vector of linear coefficients and a matrix of quadratic coefficients. We'll call these `C` and `J`.


In [3]:
# QUBO example
C = np.array([-1.0, -1.0, -1.0])
J = np.array([[0.0, 1.0, 1.0],
              [0.0, 0.0, 1.0],
              [0.0, 0.0, 0.0]])
model = QuadraticModel(C, J)
model.upper_bound = np.ones((3, )) # Set the bounds of the variablees as 1
model.qubo.Q

array([[-1. ,  0.5,  0.5],
       [ 0.5, -1. ,  0.5],
       [ 0.5,  0.5, -1. ]])

When you define a problem with integer variables that can be larger than 1, the `QuadraticModel` must translate those integers into a format that the QUBO solver can understand. 
It does this through a process called **binary expansion**.

An integer variable is represented as a sum of weighted binary variables. For an integer `x` that can go up to a bound `B`, its binary expansion would be:

\begin{equation}
x = \sum_{i=0}^{n-1} b_i \cdot 2^i = 1 \cdot b_0 + 2 \cdot b_1 + 4 \cdot b_2 + \dots
\end{equation}

In summary, the `QuadraticModel` is automatically expands integer problem into a larger, equivalent binary problem so that it can be represented as a valid QUBO matrix.

In [4]:
# QUIO example
model.upper_bound = 3*np.ones((3,)) # Set the bounds of the variables as 3
model.qubo.Q

array([[-1. ,  0. ,  0.5,  1. ,  0.5,  1. ],
       [ 0. , -2. ,  1. ,  2. ,  1. ,  2. ],
       [ 0.5,  1. , -1. ,  0. ,  0.5,  1. ],
       [ 1. ,  2. ,  0. , -2. ,  1. ,  2. ],
       [ 0.5,  1. ,  0.5,  1. , -1. ,  0. ],
       [ 1. ,  2. ,  1. ,  2. ,  0. , -2. ]], dtype=float32)

To solve the different problem types, you must selectively run the notebook cells. Please follow the instructions below for the desired model.

To Solve the QUBO Problem
For the standard QUBO problem (where variables are binary), first run `QUBO example Cell` to define the model with an upper bound of 1, and then run `Solve Problem Cell` to solve it.

Run Order: `QUBO example Cell` → `Solve Problem Cell`

To Solve the QUIO Problem
For the QUIO (integer optimization) problem with an upper bound of 3, first run `QUBO example Cell` to define the model, `QUIO example Cell` to set the upper bound of 3, and then run v solve it.

Run Order: `QUIO example Cell` → `Solve Problem Cell`

In [5]:
# Solve Problem
solver = Dirac1CloudSolver(url=api_url, api_token=api_token)
Q = model.qubo.Q
# qn = Q.shape[0]
# qubomodel = QuadraticModel(np.zeros((qn,)), Q)
# qubomodel.upper_bound = np.ones((qn,))
response = solver.solve(model) # qubomodel)
# response["results"]["solutions"], model.qubo.evaluate(np.array(response["results"]["solutions"][0]))
solution = model.decode(np.array(response["results"]["solutions"][0]), "qubo")
solution, model.evaluate(solution)

2025-08-28 22:01:57 - Dirac allocation balance = 0 s (unmetered)
2025-08-28 22:01:57 - Job submitted: job_id='68b126368060c93397963694'
2025-08-28 22:01:57 - QUEUED
2025-08-28 22:02:00 - RUNNING
2025-08-28 22:02:10 - COMPLETED
2025-08-28 22:02:13 - Dirac allocation balance = 0 s (unmetered)


(array([0, 0, 3]), -3.0)

### `PolynomialModel`

$$
E(x)=\sum_i C_i x_i + \sum_i\sum_j J_{ij}x_ix_j + \sum_i\sum_j\sum_k T_{ijk} x_ix_jx_k + \sum_i\sum_j\sum_k\sum_l Q_{ijkl} x_ix_jx_kx_l+ \sum_i\sum_j\sum_k\sum_l\sum_m P_{ijklm} x_ix_jx_kx_lx_m .
$$

### `ConstraintsMixIn`

The constraint mixin defines a standard method to convert a linear system of equality constraints into a penalty function, which is 0 for feasible solutions and positive for infeasible solutions.
$$
Ax=b\Rightarrow P(x)=(Ax-b)^2
$$

### `InequalityConstraintMixIn`

This adds a `senses` attribute which allows the definition of constraints with inequalities. These become equality constraints and are converted to penalties using the `Constraints


In [6]:
# Constraint Example
A = np.array([[1, 0, -1]]) # left-hand side of the constraint
b = np.array([0]) # right-hand side of the constraint

# Coefficients and indices of the polynomial
coeff = model.polynomial.coefficients
indices = model.polynomial.indices

# Create a new model with constraints
constraint_model = ConstraintExample(coeff, indices, A, b)
constraint_model.upper_bound = np.ones((3,)) # Upper bound of 1
constraint_model.penalty_multiplier = alpha = 2 # Penalty multiplier

# Solve the constrained problem
response = solver.solve(constraint_model)

# Get the solution and evaluate
solution = np.array(response["results"]["solutions"][0])

# Print solution and corrected objective value
np.array(solution), constraint_model.polynomial.pure_evaluate(solution) + alpha * constraint_model.offset 

2025-08-28 22:02:14 - Dirac allocation balance = 0 s (unmetered)
2025-08-28 22:02:14 - Job submitted: job_id='68b126468060c93397963695'
2025-08-28 22:02:14 - QUEUED
2025-08-28 22:02:17 - RUNNING
2025-08-28 22:02:25 - COMPLETED
2025-08-28 22:02:27 - Dirac allocation balance = 0 s (unmetered)


(array([0, 1, 0]), array([-1.]))

## Problem Type Classes

Two categories- ML and Decision Optimization.

### Decision Optimization

- `QAPModel`
- `SetPartitionModel`
- `SetCoverModel`
- `MTZTSPModel`
- `AllocationModel`
- `PortMomentum`
- `MaxCutModel`
- `GraphPartitionModel`

### Machine Learning

- `QBoostClassifier`
- `QSVMClassifier`
- `PCA`
- `LinearRegression`

### Algorithms

- `PenaltyMultiplierAlgorithm`